# 02. プロンプトエンジニアリングの基礎

このノートブックでは、「プロンプト」を工夫することでLLMの振る舞いを制御する基本的なテクニック（プロンプトエンジニアリング）を学びます。

## 事前準備

Google Colabで実行する場合、メニューの「ランタイム」→「ランタイムのタイプを変更」で、ハードウェアアクセラレータが「T4 GPU」になっていることを確認してください。

In [ ]:
# 編集禁止セル
# セットアップの確認と共通モジュールのインポート
import os
import sys
import torch
from google.colab import drive

# Google Driveのマウント
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')

# リポジトリパスの設定と移動
repo_path = '/content/llm_lab'
if not os.path.exists(repo_path):
    !git clone https://github.com/akio-kobayashi/llm_lab.git

os.chdir(repo_path)

# 必要なライブラリのインストール
!pip install -q -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft trl datasets gradio

# srcディレクトリにパスを通す
if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

# 共通関数のインポート
try:
    from src.common import load_llm, generate_text
    print('共通モジュールのインポートが完了しました。')
except ImportError:
    print('共通モジュールのインポートに失敗しました。00_setup_common.ipynb を実行したか確認してください。')

## モデルのロード

演習で共通して使用するLLMをロードします。

In [ ]:
# 編集禁止セル
model, tokenizer = None, None
try:
    model, tokenizer = load_llm(use_4bit=True)
except Exception as e:
    print(f'モデルのロード中にエラーが発生しました: {e}')

## プロンプトによる振る舞いの制御

LLMへの指示（プロンプト）に、役割（ペルソナ）や制約条件、出力形式などを具体的に記述することで、生成されるテキストをある程度コントロールすることができます。

### 演習1: 役割（ペルソナ）を与える

LLMに「あなたはプロのツアーコンダクターです」という役割を与えて、回答のスタイルを変化させてみましょう。
以下の `persona` と `question` を編集して、どのような変化が起きるか観察してください。

In [ ]:
# --- ここを編集 --- #
persona = "あなたはプロのツアーコンダクターです。利用者の要望に合わせて、丁寧かつ魅力的な旅行プランを提案してください。"
question = "家族で楽しめる、夏休みの沖縄旅行プランを考えています。"
# --- 編集ここまで --- #

# 編集禁止セル
if model and tokenizer:
    prompt = f"以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{persona}\n\n{question}\n\n### 応答:\n"
    
    generated_text = generate_text(model, tokenizer, prompt)
    
    print("--- プロンプト ---")
    print(prompt)
    print("\n--- LLMの回答 ---")
    answer = generated_text.split("### 応答:")[-1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

### 演習2: 出力形式を指示する

回答を特定のフォーマット（例：箇条書き、JSON、Markdownテーブル）で出力するように指示してみましょう。
これにより、プログラムで後処理しやすくなります。

以下の `output_format_instruction` を編集して、リストや表形式で出力させてみましょう。

In [ ]:
# --- ここを編集 --- #
question = "日本の有名なアニメ映画を5つ、監督名と公開年と合わせて教えて。"
output_format_instruction = "回答は必ず以下のMarkdownテーブル形式で出力してください。\n| 作品名 | 監督名 | 公開年 |\n|---|---|---|"
# --- 編集ここまで --- #

# 編集禁止セル
if model and tokenizer:
    prompt = f"以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{question}\n\n{output_format_instruction}\n\n### 応答:\n"
    
    generated_text = generate_text(model, tokenizer, prompt)
    
    print("--- プロンプト ---")
    print(prompt)
    print("\n--- LLMの回答 ---")
    answer = generated_text.split("### 応答:")[-1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

### 演習3: プロンプトでは知識を増やせないことの確認

プロンプトエンジニアリングは、あくまでLLMが**元々持っている知識を引き出す**ためのテクニックです。
プロンプトで指示しても、LLMが知らないことは答えられません。

`01_gpt_baseline` と同じように、LLMが知らないはずの質問をしてみましょう。
丁寧なプロンプトを与えても、結局はもっともらしい嘘（ハルシネーション）を返してしまうことを確認します。

In [ ]:
# --- ここを編集 --- #
persona = "あなたは最新のテクノロジーに精通したジャーナリストです。"
question = "Appleが来月発表すると噂の「iGlass」について、スペックを詳細に教えてください。"
output_format_instruction = "箇条書きで、スペック項目と予測される性能を記述してください。"
# --- 編集ここまで --- #

# 編集禁止セル
if model and tokenizer:
    prompt = f"以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{persona}\n\n{question}\n\n{output_format_instruction}\n\n### 応答:\n"
    
    generated_text = generate_text(model, tokenizer, prompt)
    
    print("--- プロンプト ---")
    print(prompt)
    print("\n--- LLMの回答 ---")
    answer = generated_text.split("### 応答:")[-1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

## まとめ

このノートブックでは、プロンプトエンジニアリングの基本を学びました。

- **振る舞いの制御**: ペルソナや制約を与えることで、LLMの回答スタイルを制御できる。
- **出力形式の指定**: 箇条書きやテーブル形式などを指示することで、後処理しやすい形で出力を得られる。
- **知識の限界**: プロンプトを工夫しても、LLMが元々知らない知識を創造することはできない。

LLMに外部の知識を与えて、事実に基づいた回答をさせるにはどうすればよいでしょうか？
その答えが、次の `03_rag_concept_demo.ipynb` で学ぶ **RAG (Retrieval-Augmented Generation)** です。

### メモリ解放
次のノートブックに進む前に、GPUメモリを解放します。

In [ ]:
# 編集禁止セル
import gc
if 'model' in locals() and model is not None: del model
if 'tokenizer' in locals() and tokenizer is not None: del tokenizer
model = None
tokenizer = None
gc.collect()
torch.cuda.empty_cache()
print("モデルを解放し、GPUキャッシュをクリアしました。")